# TRADES CHAINS PLOTS

In [ ]:
import numpy as np
import os
import sys

In [ ]:

from pytrades import pytrades # this import a python module, not the F90 module!
from pytrades import ancillary as anc
from pytrades import constants as cst
from pytrades.convergence import full_statistics, log_probability_trace

emcee_analysis_path = os.path.dirname(
    os.path.abspath("/path/to/trades_emcee_analysis.py")
)
sys.path.append(emcee_analysis_path)
import trades_emcee_analysis as pyta

In [ ]:
import matplotlib.pyplot as plt
# %matplotlib inline
anc.set_rcParams()

Paths

In [ ]:
exo_path = os.path.abspath(
    "/path/to/exoplanet/system/folder"
)

exo_sim = os.path.join(
    exo_path,
    "simulation_folder",
)

Read configuration file that has to have `analysis` section

In [ ]:
yml_file = os.path.join(
    exo_sim,
    "configuration.yml"
)
cli = anc.ConfigurationAnalysis(yml_file)

Change parameters of Configuration Object, such as the `thinning`:

In [ ]:
cli.use_thin = 100

Set the analysis from the Configuration Object

In [ ]:
analysis = pyta.AnalysisTRADES(cli)
conf_run = anc.ConfigurationRun(cli.yaml_file)

In [ ]:
logs_folder = os.path.join(cli.full_path, "logs")
os.makedirs(logs_folder, exist_ok=True)
plots_folder = os.path.join(cli.full_path, "plots_{}thin".format(cli.use_thin))
os.makedirs(plots_folder, exist_ok=True)

anc.print_both("\nPlotting chains/convergence ... ")

# ===== LOG-PROB TRACE plot ===== #
log_probability_trace(
    analysis.lnprobability_full_thinned, 
    analysis.lnprob_posterior, 
    plots_folder, 
    n_burn=cli.nburnin,
    n_thin=conf_run.thin_by,
    show_plot=False, 
    figsize=(6, 6)
)

# ===== CONVERGENCE ===== #
par_file = os.path.join(cli.full_path, "summary_parameters.hdf5")
stats_file = os.path.join(logs_folder, "convergence_stats.logs")
overplot = anc.set_overplot(cli.overplot)
with open(stats_file, 'w') as olog:
    with h5py.File(par_file, "r") as s_h5f:
        l = "fitted"
        if overplot is not None:
            sim_id_str = "{}".format(overplot)
            overp_par = s_h5f["parameters/{:s}/{:s}/parameters".format(sim_id_str, l)][
                ...
            ]
        else:
            overp_par = analysis.fitting_posterior[np.argmax(analysis.lnprob_posterior), :]
        exp_acf_fit, exp_steps_fit = full_statistics(
            analysis.chains_full_thinned,
            analysis.fitting_posterior,
            analysis.fitting_names,
            overp_par,
            analysis.lnprob_posterior,
            plots_folder,
            olog=olog,
            ilast=0,
            n_burn=cli.nburnin,
            n_thin=conf_run.thin_by,
            show_plot=False,
            figsize=(6, 6),
        )
        
        l = "physical"
        if overplot is not None:
            sim_id_str = "{}".format(overplot)
            overp_par = s_h5f["parameters/{:s}/{:s}/parameters".format(sim_id_str, l)][
                ...
            ]
        else:
            overp_par = analysis.physical_posterior[np.argmax(analysis.lnprob_posterior), :]
        exp_acf_phy, exp_steps_phy = full_statistics(
            analysis.physical_chains,
            analysis.physical_posterior,
            analysis.physical_names,
            overp_par,
            analysis.lnprob_posterior,
            plots_folder,
            olog=olog,
            ilast=analysis.sim.nfit,
            n_burn=cli.nburnin,
            n_thin=conf_run.thin_by,
            show_plot=False,
            figsize=(6, 6),
        )
    anc.print_both("", output=olog)
    anc.print_both(
        "All expected steps   for each parameter needed to reach full convergence:\n{}".format(
            exp_steps_fit
        ),
        output=olog,
    )
    anc.print_both(
        "All expected ACF len for each parameter needed to reach full convergence:\n{}".format(
            exp_acf_fit
        ),
        output=olog,
    )
    imax_acf = np.argmax(exp_acf_fit)
    anc.print_both(
        "MAX ACF = {} ==> needed chains of {} steps\n".format(
            exp_acf_fit[imax_acf], exp_steps_fit[imax_acf]
        ),
        output=olog,
    )